In [ ]:
import requests
import pandas  as pd
import re


To perform a GET request, use `requests.get()` and pass in the desired url.

In [ ]:
URL = 'https://realpython.github.io/fake-jobs/'

response = requests.get(URL)

Let's see what kind of object we get.

In [ ]:
type(response)

We can check the status code using the `status_code` attribute.

In [ ]:
response.status_code

In [ ]:
response.text

In [ ]:
from bs4 import BeautifulSoup as BS

In [ ]:
BS(response.text)

In [ ]:
soup = BS(response.text)

In [ ]:

title=soup.findAll('h2')
title

In [ ]:
title[0].text

In [ ]:
title_list=[]
for title in title:
    title_list.append(title.text)

In [ ]:
df=pd.DataFrame(data=title_list)
df.head()

#### c. Finally, extract the companies, locations, and posting dates for each job. For example, the first job has a company of "Payne, Roberts and Davis", a location of "Stewartbury, AA", and a posting date of "2021-04-08". Ensure that the text that you extract is clean, meaning no extra spaces or other characters at the beginning or end.  



In [ ]:
company=soup.findAll('h3')
company[0].text

In [ ]:
company_list=[]
for company in company:
    company_list.append(company.text)
company_list[:4]

In [ ]:
df['Company']=company_list
df.head(5)

In [ ]:
df=df.rename(columns={0:'Job_Title'})

df['Job_Title'].unique()

In [ ]:
df.Job_Title=df.Job_Title.str.replace('(', '', regex=False)
df.Job_Title=df.Job_Title.str.replace(')', '', regex=False)
df.Job_Title=df.Job_Title.str.replace('/', ' ', regex=False)
df.Job_Title=df.Job_Title.str.replace(',', '', regex=False)

df.head(25)

In [ ]:
locate=soup.findAll('p')
locate[0].text

In [ ]:
locations=[]
for location in locate:
    locations.append(location.text)
#locations[1:]
location_list=locations[1:]
location_list

In [ ]:
len(location_list)

In [ ]:
date_column=[]
place=[]
split_locations = [re.split(r'\+', location.strip()) for location in location_list]
for dat in split_locations:
    for string1 in dat:
      if len(string1) == 10 :
           date_column.append(string1)
      elif len(string1) !=10 :
           place.append(string1)
          
df['place']=place
df['date_added']=date_column
df



In [ ]:
link2=[]
l=soup.find_all('a', href=True)
for i in l:
    link2.append(i['href'])
link2

In [ ]:
links_list=[]
for i in link2:
    if i != 'https://www.realpython.com':
        links_list.append(i)


In [ ]:
df['apply_links']=links_list
df.head(2)

 #### 2.b. Next, get those same urls in a different way. Examine the urls and see if you can spot the pattern of how they are constructed. Then, build the url using the elements you have already extracted. Ensure that the urls that you created match those that you extracted using BeautifulSoup. Warning: You will need to do some string cleaning and prep in constructing the urls this way. For example, look carefully at the urls for the "Software Engineer (Python)" job and the "Scientist, research (maths)" job.

In [ ]:
df['jobs_link']='https://realpython.github.io/fake-jobs/jobs/'+(df['Job_Title'].str.lower().str.replace(" ","-"))+'-'+df.index.astype(str)+'.html'
df

In [ ]:
dfl=df.loc[df['apply_links'] == df['jobs_link']]


In [ ]:
dfl.shape

#### 3. Finally, we want to get the job description text for each job.  
    a. Start by looking at the page for the first job, https://realpython.github.io/fake-jobs/jobs/senior-python-developer-0.html. Using BeautifulSoup, extract the job description paragraph.  

In [ ]:
job_description=[]
for index,row in df.iterrows():
    suburl = df.loc[index,'apply_links']
    response1 = requests.get(suburl)
    soup = BS(response1.text)
    content_div = soup.find('div', class_='content')
    job = content_div.find_all('p')
    job_description.append(job[0].text)
job_description

In [ ]:
df['job_description']=job_description
df.head(3)

####   b. We want to be able to do this for all pages. Write a function which takes as input a url and returns the description text on that page. For example, if you input "https://realpython.github.io/fake-jobs/jobs/television-floor-manager-8.html" into your function, it should return the string "At be than always different American address. Former claim chance prevent why measure too. Almost before some military outside baby interview. Face top individual win suddenly. Parent do ten after those scientist. Medical effort assume teacher wall. Significant his himself clearly very. Expert stop area along individual. Three own bank recognize special good along.".  

In [ ]:
link=input ('Please input the job link : ')

In [ ]:
for index,row in df.iterrows():
  if df.loc[index,'apply_links']==link :
    print (df.loc[index,'job_description'])

#####     c. Use the [.apply method](https://pandas.pydata.org/docs/reference/api/pandas.Series.apply.html) on the url column you created above to retrieve the description text for all of the jobs.

In [ ]:
input_link=input ('Please input the job link : ')
def descb():
  for index,row in df.iterrows():
     if df.loc[index,'apply_links']==input_link :
         x=(df.loc[index,'job_description'])
  return print('\n','Job Description : ', '\n',x)

#df.apply(descb())
